En este Notebook se genera un diccionario con los nombres de las estaciones de medición de la calidad del aire en la Ciudad de Nueva York, como llaves, y las zonas tlc* que son cubiertas en el registro de contaminantes por cada estación, como valores. 

El fin de definir las zonas que cubre cada estación es para poder determinar la relación que existe entre los contaminantes emitidos por los vehículos de combustion (en este caso taxis) y la carga de circulación en un momento dado en dichas zonas. 

*Una zona TLC (Taxi and Limousine Commission) se refiere a las áreas designadas por la Comisión de Taxis y Limusinas de la Ciudad de Nueva York para regular la prestación de servicios de transporte privado en la ciudad. En el contexto del código que estás utilizando, las zonas TLC son áreas geográficas específicas dentro de la ciudad que se han definido con fines regulatorios para el transporte. Las zonas se identifican con un número de ubicación y se utilizan en el sistema de medallones de taxis y en otros servicios de transporte regulados por la Comisión de Taxis y Limusinas de la ciudad.

En la página de NYC Taxi & Limousine Commission, usando el mismo link que se utilizó para descargar la data sobre los trayectos de los taxis en la Ciudad de Nueva York: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, hay un archivo, bajo el apartado 'Taxi Zone Maps and Lookup Tables' con nombre Taxi Zone Shapefile (PARQUET) que contiene el shape file de las zonas tlc de la Ciudad de Nueva York. Descargamos este archivo y sus correspondientes archivos de referencia y los cargamos en un geoDataFrame. 

In [6]:
import pandas as pd
import geopandas as gpd
import os
os.environ['SHAPE_RESTORE_SHX'] = 'YES'

In [8]:
# Cargar el archivo shapefile
shapefile_path = 'C:\\Users\\Usuario\\Documents\\proyectos\\NYC_TAXIS\\tlc_zones\\taxi_zones.shp'
gdf = gpd.read_file(shapefile_path)

# Verificar las primeras filas de la tabla de atributos
print(gdf.head())

# Verificar el sistema de coordenadas del archivo
print(gdf.crs)

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((933100.918 192536.086, 933091.011 19...  
1         Queens  MULTIPOLYGON (((1033269.244 172126.008, 103343...  
2          Bronx  POLYGON ((1026308.770 256767.698, 1026495.593 ...  
3      Manhattan  POLYGON ((992073.467 203714.076, 992068.667 20...  
4  Staten Island  POLYGON ((935843.310 144283.336, 936046.565 14...  
EPSG:2263


Las coordenadas que aparecen en la columna 'geometry' del GeoDataFrame son coordenadas en proyección cartográfica del sistema de referencia EPSG:2263, que es una proyección plana con unidades en pies, utilizada para representar datos cartográficos de la ciudad de Nueva York. En este sistema, las coordenadas representan la distancia en pies desde un origen en la esquina suroeste del área de interés.

Dado que las coordenadas que tenemos para las ubicaciones de las estaciones de medición de contaminantes se encuentran en grados, hay realizar primero una conversión a epsg:4326 que es un sistema de coordenadas en grados. 

In [9]:
# Transformar las coordenadas a EPSG:4326
gdf = gdf.to_crs(epsg=4326)

In [10]:
# Verificar las primeras filas de la tabla de atributos
print(gdf.head())

# Verificar el sistema de coordenadas del archivo
print(gdf.crs)

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((-74.18445 40.69500, -74.18449 40.695...  
1         Queens  MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...  
2          Bronx  POLYGON ((-73.84793 40.87134, -73.84725 40.870...  
3      Manhattan  POLYGON ((-73.97177 40.72582, -73.97179 40.725...  
4  Staten Island  POLYGON ((-74.17422 40.56257, -74.17349 40.562...  
EPSG:4326


In [11]:
# Guardamos el archivo con las coordenadas convertidas
gdf.to_file('nyc_taxi_zones_wgs84.shp', driver='ESRI Shapefile')

Ahora, cómo la medición de las partículas PM 2.5 tiene un alcance de 4 km, queremos saber qué zonas de la ciudad de Nueva York se encuentran dentro del radio de medición de cada estación. 

Para ello utilizaremos la distinacia haversiana para calcular, para cada punto de cada polígono creado para cada zona tlc que considera la NYC Taxi & Limousine Commission para registrar los recorridos de los taxis.

Después se agregarán a una lista (por estación de medición) las zonas para las que al menos uno de sus puntos de polígono se encuentren dentro del radio de 2km de la estación y se añadirá qué porcentaje de los puntos del polígono que componen la zona se encuentrán dentro de dicho radio, para posteriormente poder tomar una decisión respecto a qué zonas considerar para relacionar con la medición de contaminación de cada estación. 

In [27]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Point

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000 # radio de la Tierra en metros
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)

    a = sin(delta_phi / 2)**2 + cos(phi1) * cos(phi2) * sin(delta_lambda / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

# Cargar el archivo shapefile
shapefile_path = 'nyc_taxi_zones_wgs84.shp'
gdf = gpd.read_file(shapefile_path)

# DataFrame de estaciones
estaciones = pd.DataFrame({
    'Station': ['IS 143', 'Morrisania', 'CCNY', 'Queens Near Road', 'Queens College', 'Maspeth', 'PS 274', 'PS 314', 'Port Richmond', 'Fresh Kills West'],
    'Latitude': [40.848913, 40.830780, 40.819732, 40.739264, 40.736251, 40.726980, 40.694540, 40.641820, 40.633060, 40.580254],
    'Longitude': [-73.930604, -73.920059, -73.948239, -73.817694, -73.821655, -73.893120, -73.927690, -74.018710, -74.137156, -74.198295]
})

# Lista para almacenar los resultados
estaciones_con_location_ids = []

# Radio en metros
radius = 2000

for i, estacion in estaciones.iterrows():
    estacion_coords = Point(estacion['Longitude'], estacion['Latitude'])
    estacion_name = estacion['Station']
    estacion_location_ids = []

    for j, zone in gdf.iterrows():
        if zone['geometry'].contains(estacion_coords):
            estacion_location_ids.append(zone['LocationID'])
        else:
            num_points_inside_radius = 0
            if zone['geometry'].geom_type == 'Polygon':
                # Si la geometría es un polígono simple
                for point in zone['geometry'].exterior.coords:
                    if haversine_distance(estacion_coords.y, estacion_coords.x, point[1], point[0]) <= radius:
                        num_points_inside_radius += 1
                if num_points_inside_radius > 0:
                    percentage_inside_radius = round((num_points_inside_radius / len(zone['geometry'].exterior.coords)) * 100, 2)
                    estacion_location_ids.append((zone['LocationID'], percentage_inside_radius))
            elif zone['geometry'].geom_type == 'MultiPolygon':
                # Si la geometría es un multipolígono
                for polygon in zone['geometry'].geoms:
                    # Iterar sobre cada polígono en el multipolígono
                    num_points_inside_radius_polygon = 0
                    for point in polygon.exterior.coords:
                        if haversine_distance(estacion_coords.y, estacion_coords.x, point[1], point[0]) <= radius:
                            num_points_inside_radius_polygon += 1
                    if num_points_inside_radius_polygon > 0:
                        percentage_inside_radius_polygon = round((num_points_inside_radius_polygon / len(polygon.exterior.coords)) * 100, 2)
                        estacion_location_ids.append((zone['LocationID'], percentage_inside_radius_polygon))
                    # Iterar sobre cada interior en el polígono, si existe
                    for interior in polygon.interiors:
                        num_points_inside_radius_interior = 0
                        for point in interior.coords:
                            if haversine_distance(estacion_coords.y, estacion_coords.x, point[1], point[0]) <= radius:
                                num_points_inside_radius_interior += 1
                        if num_points_inside_radius_interior > 0:
                            percentage_inside_radius_interior = round((num_points_inside_radius_interior / len(interior.coords)) * 100, 2)
                            estacion_location_ids.append((zone['LocationID'], percentage_inside_radius_interior))

    # Añadir la estación y los location_ids a la lista de resultados
    estaciones_con_location_ids.append({
        'Station': estacion_name,
        'Location IDs': estacion_location_ids
    })


In [28]:
estaciones_con_location_ids

[{'Station': 'IS 143',
  'Location IDs': [(42, 28.97),
   (69, 18.08),
   (119, 76.81),
   (120, 90.2),
   (127, 57.61),
   (127, 100.0),
   (128, 0.16),
   (136, 6.25),
   (169, 41.98),
   (235, 67.32),
   243,
   (244, 81.42),
   (247, 40.31)]},
 {'Station': 'Morrisania',
  'Location IDs': [(42, 81.0),
   (47, 49.32),
   (59, 24.29),
   (60, 3.57),
   69,
   (74, 7.31),
   (116, 14.95),
   (119, 100.0),
   (120, 57.98),
   (147, 25.37),
   (159, 84.56),
   (167, 79.39),
   (168, 10.75),
   (169, 43.21),
   (235, 34.63),
   (244, 26.23),
   (247, 91.62)]},
 {'Station': 'CCNY',
  'Location IDs': [(41, 50.72),
   (42, 90.65),
   (69, 10.17),
   (74, 44.52),
   (116, 100.0),
   (119, 23.19),
   (120, 22.13),
   152,
   (159, 4.03),
   (166, 67.57),
   (168, 14.93),
   (244, 26.78),
   (247, 42.41)]},
 {'Station': 'Queens Near Road',
  'Location IDs': [(9, 23.81),
   (73, 52.69),
   (92, 44.75),
   (93, 18.52),
   (98, 5.08),
   (121, 60.27),
   (135, 72.58),
   192]},
 {'Station': 'Queen